# Grafana POC Demo: Safety Gear Detection


This is a sample proof of concept demo for Intel Metering and Grafana integration using the safety gear detection demo.
The demo is currently not available for all users.

This demo assume that you have already gone through the Python safety gear detection example.


## Step 0: Set Up

### 0.1: Import dependencies

Run the below cell to import Python dependencies needed for displaying the results in this notebook
(tip: select the cell and use **Ctrl+enter** to run the cell)

In [1]:
from IPython.display import HTML
import matplotlib.pyplot as plt
import os
import time
import sys
from pathlib import Path
sys.path.insert(0, str(Path().resolve().parent.parent))
from demoTools.demoutils import *

### 0.2  (Optional-step): Original video without inference

If you are curious to see the input video, run the following cell to view the orignal video stream used for inference and object detection.

In [2]:
!ln -sf /data/reference-sample-data/safety-gear-detection/Safety_Full_Hat_and_Vest.mp4
videoHTML('Workers video', ['Safety_Full_Hat_and_Vest.mp4'])

## Step 1: Creating the models

Just as in the main safety gear demo, we will use an existing Caffe model specially trained for detecting people, hardhats and safety vests.
Let's convert the model using the model optimizer.

In [3]:
!/opt/intel/computer_vision_sdk/deployment_tools/model_optimizer/mo.py \
--input_model /data/reference-sample-data/safety-gear-detection/worker_safety_mobilenet.caffemodel \
--model_name mobilenet-ssd \
--data_type FP32 \
-o models/mobilenet-ssd/FP32

Model Optimizer arguments:
Common parameters:
	- Path to the Input Model: 	/data/reference-sample-data/safety-gear-detection/worker_safety_mobilenet.caffemodel
	- Path for generated IR: 	/home/u19887/Reference-samples/iot-devcloud/grafana/safety-gear-detection-python/models/mobilenet-ssd/FP32
	- IR output name: 	mobilenet-ssd
	- Log level: 	ERROR
	- Batch: 	Not specified, inherited from the model
	- Input layers: 	Not specified, inherited from the model
	- Output layers: 	Not specified, inherited from the model
	- Input shapes: 	Not specified, inherited from the model
	- Mean values: 	Not specified
	- Scale values: 	Not specified
	- Scale factor: 	Not specified
	- Precision of IR: 	FP32
	- Enable fusing: 	True
	- Enable grouped convolutions fusing: 	True
	- Move mean values to preprocess section: 	False
	- Reverse input channels: 	False
Caffe specific parameters:
	- Enable resnet optimization: 	True
	- Path to the Input prototxt: 	/data/reference-sample-data/safety-gear-detection/worke

**Note** the above line is a single command line input, which spans 4 lines thanks to the backslash '\\', which is a line continuation character in Bash.

Here, the arguments are:
* --input-model : the original model
* --data_type : Data type to use. One of {FP32, FP16, half, float}
* -o : outout dirctory

This script also supports `-h` that will you can get the full list of arguments.

With the `-o` option set as above, this command will write the output to the directory `models/mobilenet-ssd/FP32`

There are two files produced:
```
models/mobilenet-ssd/FP32/mobilenet-ssd.xml
models/mobilenet-ssd/FP32/mobilenet-ssd.bin
```
These will be used later in the exercise.

## Step 2 : Meteraing for Inference on a video

Statistic collection is done on the compute nodes using colectd service with Intel Metering module enabled. In this section, we will submit a job to the queue and log the usage statistics to a file remotely.

### 2.1 Creating job file

The collectd statistics can be accessed using the `curl` command.
For example, here is the curl command for the last 100 seconds of cpu usage.

```
curl "http://127.0.0.1/render?target=Collectdc003-n014.cpu*.*&from=-100seconds&format=json" > ~/Metering/$PBS_JOBID.CPU.json
```

In the bellow job script, we query information from collectd at the end of the script. We measure the time it took for the python program (initialization included), then gather statistics by looking back for the same duration.
Run this cell to create the job script.

In [23]:
%%writefile object_detection_job.sh

# The default path for the job is your home directory, so we change directory to where the files are.
cd $PBS_O_WORKDIR

# Object detection script writes output to a file inside a directory. We make sure that this directory exists.
# The output directory is the first argument of the bash script
OUTPUT_FILE=$1
DEVICE=$2
FP_MODEL=$3
INPUT_FILE=$4

if [ "$2" = "HETERO:FPGA,CPU" ]; then
    # Environment variables and compilation for edge compute nodes with FPGAs
    export LD_LIBRARY_PATH=${LD_LIBRARY_PATH}:/opt/altera/aocl-pro-rte/aclrte-linux64/
    source /opt/fpga_support_files/setup_env.sh
    aocl program acl0 /opt/intel/computer_vision_sdk/bitstreams/a10_vision_design_bitstreams/5-0_PL1_FP11_MobileNet_Clamp.aocx
fi

STIME=`date +%s`
# Running the object detection code
SAMPLEPATH=$PBS_O_WORKDIR
python3 object_detection_demo_ssd_async.py -m ${SAMPLEPATH}/models/mobilenet-ssd/${FP_MODEL}/mobilenet-ssd.xml \
                                           -i $INPUT_FILE \
                                           -o $OUTPUT_FILE \
                                           -d $DEVICE \
                                           -l /data/reference-sample-data/extension/libcpu_extension.so \
                                           --labels /data/reference-sample-data/safety-gear-detection/labels.txt
ETIME=`date +%s`
let LOOKBACK_TIME=ETIME-STIME
curl "http://127.0.0.1:80/render?target=Collectdc003-n014.cpu*.*&from=-"$LOOKBACK_TIME"seconds&format=json" > ~/Metering/$PBS_JOBID.CPU.json
curl "http://127.0.0.1:80/render?target=Collectdc003-n014.Metering_plugin.renderbusy&from=-"$LOOKBACK_TIME"seconds&format=json" > ~/Metering/$PBS_JOBID.GPU.json
curl "http://127.0.0.1:80/render?target=Collectdc003-n014.memory.*&from=-"$LOOKBACK_TIME"seconds&format=json" > ~/Metering/$PBS_JOBID.MEMORY.json
curl "http://127.0.0.1:80/render?target=Collectdc003-n014.thermal-thermal*.temperature&from=-"$LOOKBACK_TIME"seconds&format=json" > ~/Metering/$PBS_JOBID.THERMAL.json

Overwriting object_detection_job.sh


Note that we are only running the inference. But we are not rendering the video, as we are just interesrted in the inference part of the workload.

Here, the properties describe the node, and number on the left is the number of available nodes of that architecture.

### 2.3 Job queue submission

**Note** If you want to use your own video, Change the environment variable 'VIDEO' in the following cell from "/data/reference-sample-data/safety-gear-detection/Safety_Full_Hat_and_Vest.mp4" to the full path of your uploaded video.

In [5]:
os.environ["VIDEO"] = "/data/reference-sample-data/safety-gear-detection/Safety_Full_Hat_and_Vest.mp4"

Run the following cell to submit the above job. Currently, only one node is supported.
This node is a <a href="https://software.intel.com/en-us/iot/hardware/iei-tank-dev-kit-core">IEI 
    Tank 870-Q170</a> edge node with an <a 
    href="https://ark.intel.com/products/88178/Intel-Xeon-Processor-E3-1268L-v5-8M-Cache-2-40-GHz-">Intel 
    Xeon Processor E3-1268L v5</a>. The inference workload will run on the CPU.

In [21]:
#Submit job to the queue
job_id_core = !qsub object_detection_job.sh -l nodes=c003-n014 -F "results/ CPU FP32 $VIDEO" -N obj_det_xeon
print(job_id_core[0]) 
#Progress indicators
if job_id_core:
    progressIndicator('results/', 'i_progress_'+job_id_core[0]+'.txt', "Inference", 0, 100)

6062.c003


#### Submitting to an edge compute node with Intel Xeon CPU and using the onboard Intel GPU
In the cell below, we submit a job to the same system. But in this case the inference workload will be run on the onboard Intel GPU.

In [22]:
#Submit job to the queue
job_id_gpu = !qsub object_detection_job.sh -l nodes=c003-n014 -F "results/ GPU FP32 $VIDEO" -N obj_det_gpu 
print(job_id_gpu[0]) 
#Progress indicators
if job_id_gpu:
    progressIndicator('results/', 'i_progress_'+job_id_gpu[0]+'.txt', "Inference", 0, 100)

6063.c003


### 2.4 Check if the jobs are done

To check on the jobs that were submitted, use the `qstat` command.

We have created a custom Jupyter widget  to get live qstat update.
Run the following cell to bring it up. 

In [8]:
liveQstat()

Output(layout=Layout(border='1px solid gray', width='100%'))

Button(description='Stop', style=ButtonStyle())

You should see the jobs you have submitted (referenced by `Job ID` that gets displayed right after you submit the job in step 2.3).
There should also be an extra job in the queue "jupyterhub": this job runs your current Jupyter Notebook session.

The 'S' column shows the current status. 
- If it is in Q state, it is in the queue waiting for available resources. 
- If it is in R state, it is running. 
- If the job is no longer listed, it means it is completed.

**Note**: Time spent in the queue depends on the number of users accessing the edge nodes. Once these jobs begin to run, they should take from 1 to 5 minutes to complete. 

***Wait!***

Please wait for the inference jobs to complete before proceeding to the next step.

## Step 3: View Results

Once the jobs are completed, the results are automatically uploaded to Grafana server at <a href="http://grafana.colfaxresearch.com" >grafana.colfaxresearch.com</a>. The automation will create a dashboard for you with the name {User ID}.{Job ID} 